In [1]:
from Crypto.Cipher import AES

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]

In [2]:
key = b"YELLOW SUBMARINE"
ECBcipher = AES.new(key, AES.MODE_ECB)

In [3]:
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * ( npad % blocksize )

def XOR(X: bytes, Y: bytes) -> bytes:
    assert type(X) is bytes and type(Y) is bytes and len(X) == len(Y)
    return bytes([ x^y for x, y in zip(X, Y) ])

def EncryptCBC(plaintext, key, initialization):
    BLOCKSIZE = 16
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    plain_blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cipher_blocks = [None] * len(plain_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = XOR(plain_blocks[n], initialization)
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
        else:
            plain_block = XOR(plain_blocks[n], cipher_blocks[n-1])
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
    
    return b''.join(cipher_blocks)

In [4]:
initialization = bytes([0] * 16)
plaintext = b'''
But, soft! what light through yonder window breaks? It is the east, and Juliet is the sun.
Arise, fair sun, and kill the envious moon,
Who is already sick and pale with grief,
That thou her maid art far more fair than she. It is my lady, O, it is my love!
O, that she knew she were!
She speaks, yet she says nothing: what of that? Her eye discourses; I will answer it.
I am too bold, 'tis not to me she speaks:
Two of the fairest stars in all the heaven,
Having some business, do entreat her eyes
To twinkle in their spheres till they return.
What if her eyes were there, they in her head?
The brightness of her cheek would shame those stars, As daylight doth a lamp; her eyes in heaven
Would through the airy region stream so bright That birds would sing and think it were not night. 
See how she leans her cheek upon her hand!
O, that I were a glove upon that hand,
That I might touch that cheek!
'''

ciphertext = EncryptCBC(plaintext, key, initialization)
print(ciphertext[:100])

b'`\x95\xee\xc2w(\xaau0w$\xa0\x10\x08\x8d\x95*\xa1t\x9a.(u\xf9#*cb\xeaU\x1d\x89\xa5<\xe3\xcc\x01\xdf\x02\xa8\x1d\x1f5\x95;\xd9.\x9be\x15\x1b\x10\xcc\\"\xa9\xd1_*+\x85\xff\xb2\x90\x83M-C\xb6O\xf5{\xd4\x9e\xfc\xb0\x1fx\x8b5[QX2\xc0v\x9d\xed\x13?\xb9\x1c\x0b\xf8f!\x87\xfa\xff\xfb'


In [5]:
def DecryptCBC(ciphertext, key, initialization):
    BLOCKSIZE = 16
    assert len(ciphertext) % BLOCKSIZE == 0
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    cipher_blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
    plain_blocks = [None] * len(cipher_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, initialization)
        else:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, cipher_blocks[n-1])
            
    plaintext = b''.join(plain_blocks)
    last = plaintext[-1]
    
    if all( char == last for char in plaintext[-last:] ):
        return plaintext[:-last]
    else:
        return plaintext
        

In [6]:
print(DecryptCBC(ciphertext, key, initialization))

b"\nBut, soft! what light through yonder window breaks? It is the east, and Juliet is the sun.\nArise, fair sun, and kill the envious moon,\nWho is already sick and pale with grief,\nThat thou her maid art far more fair than she. It is my lady, O, it is my love!\nO, that she knew she were!\nShe speaks, yet she says nothing: what of that? Her eye discourses; I will answer it.\nI am too bold, 'tis not to me she speaks:\nTwo of the fairest stars in all the heaven,\nHaving some business, do entreat her eyes\nTo twinkle in their spheres till they return.\nWhat if her eyes were there, they in her head?\nThe brightness of her cheek would shame those stars, As daylight doth a lamp; her eyes in heaven\nWould through the airy region stream so bright That birds would sing and think it were not night. \nSee how she leans her cheek upon her hand!\nO, that I were a glove upon that hand,\nThat I might touch that cheek!\n"


In [7]:
import base64

with open('ciphertext.txt', mode = 'rb') as file:
    ciphertext = base64.decodebytes(file.read())
    
plaintext = DecryptCBC(ciphertext, key, initialization)
print(plaintext)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 